In [3]:
import itertools
from scipy.special import comb
import numpy as np
from collections import Counter
import math

In [4]:
t = 2
k = t+1
v = 9
b = int(comb(v, t) / (t+1)) # when k = t+1
lambda_2 = 1 # when t = 2
lambda_1 = int(b*k/v)
print(b, lambda_2, lambda_1)

12 1 4


In [5]:
%%time
combs = list ( itertools.combinations(range(1, 1+v), k) )

CPU times: user 9 µs, sys: 7 µs, total: 16 µs
Wall time: 19.6 µs


In [6]:
int(comb(int(comb(v, k)), b))

112992892764569

what about building up iteratively?

In [7]:
# choose lambda_1 of 1s...
# then count 2s, add enough until reach lambda_1 of 2s...
# if too many, then remove them from the list.
# repeat until v

In [8]:
combs = np.array(combs)

In [9]:
to_choose = []
for i in range(1, v+1):
    vals = combs[np.any(combs == i, axis=1)]
    vals = vals[np.all(vals >= i, axis=1)]
    print(i, len(vals))
    to_choose.append(vals)

1 28
2 21
3 15
4 10
5 6
6 3
7 1
8 0
9 0


In [10]:
print(len(to_choose[0]), lambda_1)
print(comb(len(to_choose[0]), lambda_1))
with_1s = np.array(list(itertools.combinations(to_choose[0], lambda_1)))
for i in range(2, v+1):
    mask = np.sum(np.any(with_1s == i, axis=2), axis=1) == lambda_2
    with_1s = with_1s[mask]
print(len(with_1s))

28 4
20475.0
105


In [11]:
plus_2s = np.array(list(itertools.combinations(to_choose[1], lambda_1 - lambda_2)))
print(len(plus_2s))
for i in range(3, v+1):
    mask = np.sum(np.any(plus_2s == i, axis=2), axis=1) <= lambda_2
    plus_2s = plus_2s[mask]
print(len(plus_2s))

1330
105


In [12]:
with_2s = np.array([np.concatenate(i) for i in itertools.product(with_1s, plus_2s)])

In [13]:
len(with_2s)

11025

In [14]:
for i in range(3, v+1):
    mask = np.sum(np.sum(with_2s == 2, axis=2) + np.sum(with_2s == i, axis=2) == 2, axis=1) == 1
    with_2s = with_2s[mask]
print(len(with_2s))

1575


In [15]:
for i in range(3, v+1):
    for j in range(i+1, v+1):
        mask = np.sum(np.sum(with_2s == i, axis=2) + np.sum(with_2s == j, axis=2) == 2, axis=1) <= 1
        with_2s = with_2s[mask]
print(len(with_2s))

840


In [16]:
with_2s[0]

array([[1, 2, 3],
       [1, 4, 5],
       [1, 6, 7],
       [1, 8, 9],
       [2, 4, 6],
       [2, 5, 8],
       [2, 7, 9]])

In [17]:
# guess: the first one will work...

In [18]:
num_remaining = b - 2*lambda_1 + 1
print(num_remaining)

5


In [19]:
remaining_opts = [item for sublist in to_choose[2:] for item in sublist]

In [20]:
len(remaining_opts)

35

In [21]:
remainders = np.array(list(itertools.combinations(remaining_opts, num_remaining)))
for i in range(3, v+1):
    for j in range(i+1, v+1):
        mask = np.sum(np.sum(remainders == i, axis=2) + np.sum(remainders == j, axis=2) == 2, axis=1) <= 1
        remainders = remainders[mask]
print(len(remainders))

1050


In [22]:
remainders[0]

array([[3, 4, 5],
       [3, 6, 7],
       [3, 8, 9],
       [4, 6, 8],
       [4, 7, 9]])

In [23]:
to_test = np.array([np.concatenate(i) for i in itertools.product(with_2s, remainders)])

In [24]:
print(len(to_test))

882000


In [25]:
to_test[0]

array([[1, 2, 3],
       [1, 4, 5],
       [1, 6, 7],
       [1, 8, 9],
       [2, 4, 6],
       [2, 5, 8],
       [2, 7, 9],
       [3, 4, 5],
       [3, 6, 7],
       [3, 8, 9],
       [4, 6, 8],
       [4, 7, 9]])

In [26]:
for i in range(1, v+1):
    for j in range(i+1, v+1):
        mask = np.sum(np.sum(to_test == i, axis=2) + np.sum(to_test == j, axis=2) == 2, axis=1) == lambda_2
        to_test = to_test[mask]
print(len(to_test))

840


In [27]:
to_test[0]

array([[1, 2, 3],
       [1, 4, 5],
       [1, 6, 7],
       [1, 8, 9],
       [2, 4, 6],
       [2, 5, 8],
       [2, 7, 9],
       [3, 4, 9],
       [3, 5, 7],
       [3, 6, 8],
       [4, 7, 8],
       [5, 6, 9]])

In [28]:
# you have 3 1's
# you have 3 2's
# you have 1 or 2 3's but you need 3 = lambda_1 .. o 

## another attempt: random

In [139]:
import random

In [146]:
tests = [random.sample(combs, b) for _ in range(200000)]

In [147]:
%%time
xs = [np.sum(np.array(tests) == i, axis=2) for i in range(1, 1+v)]

CPU times: user 7.78 s, sys: 67.9 ms, total: 7.85 s
Wall time: 7.84 s


In [148]:
for i in range(v):
    print(xs[0].shape)
    mask = np.sum(xs[i] == 1, axis=1) == lambda_1
    xs = [x[mask, ] for x in xs]

(200000, 12)
(51652, 12)
(13123, 12)
(3345, 12)
(880, 12)
(212, 12)
(69, 12)
(22, 12)
(9, 12)


In [149]:
ij_opts = list(itertools.combinations(range(v), 2))
for i, j in ij_opts:
    print(xs[0].shape)
    mask = np.sum(xs[i] + xs[j] == 2, axis=1) == lambda_2
    xs = [x[mask, ] for x in xs]

(9, 12)
(6, 12)
(3, 12)
(1, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)
(0, 12)


## Third attempt

Split out balance check

In [14]:
%%time
xs = [np.sum(np.array(opts) == i, axis=2) for i in range(1, 1+v)]

CPU times: user 14.3 s, sys: 6.75 s, total: 21.1 s
Wall time: 9.53 s


In [15]:
for i in range(v):
    print(xs[0].shape)
    mask = np.sum(xs[i] == 1, axis=1) == lambda_1
    xs = [x[mask, ] for x in xs]

(6724520, 7)
(2204475, 7)
(721725, 7)
(237763, 7)
(79842, 7)
(28055, 7)
(11205, 7)


In [16]:
ij_opts = list(itertools.combinations(range(v), 2))
for i, j in ij_opts:
    print(xs[0].shape)
    mask = np.sum(xs[i] + xs[j] == 2, axis=1) == lambda_2
    xs = [x[mask, ] for x in xs]

(11205, 7)
(6675, 7)
(4005, 7)
(2442, 7)
(1557, 7)
(1125, 7)
(1125, 7)
(639, 7)
(360, 7)
(207, 7)
(135, 7)
(135, 7)
(90, 7)
(63, 7)
(51, 7)
(51, 7)
(36, 7)
(30, 7)
(30, 7)
(30, 7)
(30, 7)


## Second attempt

Just check pairs immediately with numpy

In [16]:
%%time
opts = np.array(opts)

CPU times: user 16.9 s, sys: 110 ms, total: 17 s
Wall time: 16.9 s


In [17]:
%%time
xs = [np.sum(np.array(opts) == i, axis=2) for i in range(1, 1+v)]

CPU times: user 13.9 s, sys: 7.06 s, total: 21 s
Wall time: 9.01 s


In [18]:
ij_opts = list(itertools.combinations(range(v), 2))

In [19]:
for i, j in ij_opts:
    print(xs[0].shape)
    mask = np.sum(xs[i] + xs[j] == 2, axis=1) == 1
    xs = [x[mask, ] for x in xs]

(6724520, 7)
(2968875, 7)
(1282710, 7)
(541926, 7)
(225967, 7)
(84265, 7)
(72675, 7)
(32340, 7)
(14040, 7)
(5973, 7)
(2328, 7)
(2025, 7)
(882, 7)
(371, 7)
(144, 7)
(123, 7)
(60, 7)
(30, 7)
(30, 7)
(30, 7)
(30, 7)


## First attempt

Make sure all numbers are balanced with counters

In [14]:
# first attempt: Counters
# corr = []
# for idx, o in enumerate(opts):
#     if idx % 100000 == 0:
#         print(idx)
#     c = sum([Counter(i) for i in o], Counter())
#     if len(set(c.values())) == 1:
#         corr.append(o)

In [15]:
# go through combinations 
# good = []
# ij_opts = list(itertools.combinations(range(7), 2))
# for c in corr:
#     c_np = np.array(c)
#     ps = [np.sum(c_np == i, axis=1) for i in range(1, 1+7)]
#     ok = True
#     for i, j in ij_opts:
#         if sum(ps[i] + ps[j] == 2) != 1:
#             ok = False
#             break

#     if ok:
#         good.append(c)
# print(len(good))

To check the pairs

In [226]:
xs = [np.sum(np.array(corr) == i, axis=2) for i in range(1, 1+7)]
for i, j in ij_opts:
    print(xs[0].shape)
    mask = np.sum(xs[i] + xs[j] == 2, axis=1) == 1
    xs = [x[mask, ] for x in xs]

(11205, 7)
(6675, 7)
(4005, 7)
(2442, 7)
(1557, 7)
(1125, 7)
(1125, 7)
(639, 7)
(360, 7)
(207, 7)
(135, 7)
(135, 7)
(90, 7)
(63, 7)
(51, 7)
(51, 7)
(36, 7)
(30, 7)
(30, 7)
(30, 7)
(30, 7)
